In [1]:
import pandas as pd
import numpy as np
import pickle
from surprise import SVD, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

In [2]:
reader = Reader()
ratings = pd.read_csv('ratings_ordered.csv')
ratings.head()

,userId,rating,movieId
0,270896,5.0,110
1,270896,5.0,858
2,270896,5.0,595
3,270896,5.0,364
4,270896,5.0,4995


In [3]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [4]:
collabModel = SVD()

In [12]:
cv=5
testResults = cross_validate(collabModel, data, measures=['rmse', 'mae'], cv=cv)
for numResult in range(cv):
    print("Conjunto", numResult+1)
    print("RECM (raíz del error cuadrático medio): ", testResults["test_rmse"][numResult])
    print("EAM (error absoluto medio): ", testResults["test_mae"][numResult])
    print("-------------------------------------------")

Conjunto 1
RECM (raíz del error cuadrático medio):  0.9168083296989601
EAM (error absoluto medio):  0.7050180991098368
-------------------------------------------
Conjunto 2
RECM (raíz del error cuadrático medio):  0.905853563352153
EAM (error absoluto medio):  0.6993667542728259
-------------------------------------------
Conjunto 3
RECM (raíz del error cuadrático medio):  0.907502535936511
EAM (error absoluto medio):  0.7014372394396456
-------------------------------------------
Conjunto 4
RECM (raíz del error cuadrático medio):  0.9174502559557616
EAM (error absoluto medio):  0.7066797761817417
-------------------------------------------
Conjunto 5
RECM (raíz del error cuadrático medio):  0.913575438708722
EAM (error absoluto medio):  0.7051810623928156
-------------------------------------------


In [6]:
data_train = data.build_full_trainset()
collabModel.fit(data_train)

In [7]:
ratings[ratings['userId'] == 270896]

,userId,rating,movieId
0,270896,5.0,110
1,270896,5.0,858
2,270896,5.0,595
3,270896,5.0,364
4,270896,5.0,4995
...,...,...,...
97,270896,2.5,95
98,270896,2.5,786
99,270896,2.5,344
100,270896,2.0,19


In [14]:
userID = 270896
moviesIDs = [110, 4995, 19, 70, 64]
for movieID in moviesIDs:
    puntReal = ratings[np.logical_and(ratings['userId'] == userID, ratings['movieId'] == movieID)]["rating"].values
    if puntReal.size > 0:
        print("Puntuación real para la película {} del usuario {}: {:.2f}".format(movieID, userID, puntReal[0]))
    else:
        print("Película aún sin valorar por el usuario")
    print("Puntuación predicha para la película {} del usuario {}: {:.2f}\n".format(movieID, userID, collabModel.predict(userID, movieID).est))

Puntuación real para la película 110 del usuario 270896: 5.00
Puntuación predicha para la película 110 del usuario 270896: 4.16

Puntuación real para la película 4995 del usuario 270896: 5.00
Puntuación predicha para la película 4995 del usuario 270896: 4.52

Puntuación real para la película 19 del usuario 270896: 2.00
Puntuación predicha para la película 19 del usuario 270896: 2.73

Película aún sin valorar por el usuario
Puntuación predicha para la película 70 del usuario 270896: 3.66

Película aún sin valorar por el usuario
Puntuación predicha para la película 64 del usuario 270896: 3.18



In [9]:
fileTable = open("collabModel.pkl", "wb")
pickle.dump(collabModel, fileTable)
fileTable.close()

In [10]:
fileTable = open("collabModel.pkl", "rb")
collabModelLoad = pickle.load(fileTable)
fileTable.close()
collabModelLoad.predict(270896, 110).est

4.183435618816618